In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pyroSAR
from s1pro.s1_coh_proc import S1_coh_proc
from s1pro.s1_simple_proc import S1_simple_proc
from spatialist import Vector
import geopandas as gpd
# import logging

# detailed debug info
# logging.basicConfig(level=logging.DEBUG)

In [3]:
# aoi = Vector('/data/aoi_bretagne.geojson')
infiles = [
    "/data/S1/S1A_IW_SLC__1SDV_20230826T180527_20230826T180554_050050_0605B2_E768.zip",
    "/data/S1/S1A_IW_SLC__1SDV_20230814T180526_20230814T180553_049875_05FFB4_7D92.zip",
]
file_aoi = "/data/aoi_bretagne.geojson"
# gpt_paras = ["-e", "-x", "-c","35G", "-q", "4", "-J-Xms25G", "-J-Xmx75G"]
# gpt_paras = ["-e", "-x", "-c", "4G", "-q", "6", "-J-Xms4G", "-J-Xmx10G", "-Dsnap.gpf.disableTileCache=true"]
# gpt_paras = ["-x", "-c", "8G", "-q", "6", "-J-Xms1G", "-J-Xmx10G"]
# gpt_paras = ["-x", "-c", "8027m", "-q", "6", "-J-Xmx11468m",  "-J-Xms2048m", "-J-XX:+AggressiveOpts", "-J-Xverify:none"]
# gpt_paras = ["-x", "-c", "8027m", "-q", "8", "-J-Xmx20G",  "-J-Xms2048m", "-J-XX:+AggressiveOpts", "-J-Xverify:none"]
# S1_coh_proc(infiles, out_dir='/data/res', shapefile=aoi, pol='VV',tmpdir='/data/tmp', gpt_paras=gpt_paras)
# S1_coh_proc(infiles, out_dir='/data/res', shapefile=aoi, pol='VV',tmpdir='/data/tmp')
# S1_coh_proc(infiles, out_dir='/data/res', IWs=['IW3'], pol='VV',tmpdir='/data/tmp', gpt_paras=gpt_paras)
# S1_coh_proc(infiles, out_dir='/data/res',
#             tmpdir='/data/tmp',
#             pol='VV',
#             IWs='IW1',
#             gpt_paras=gpt_paras)
# info = pyroSAR.identify_many(infiles)
# for i in info:
# print(i)
df_aoi = gpd.read_file(file_aoi)

In [5]:
S1_simple_proc(
    infiles[0],
    infiles[1],
    out_dir="/data/res",
    tmp_dir="/data/tmp",
    shp=df_aoi.geometry[0],
    pol="VV",
)

DEBUG:pyroSAR.snap.auxil:writing /data/tmp/graph_coh.xml
INFO:pyroSAR.snap.auxil:executing node sequences..
DEBUG:pyroSAR.snap.auxil:creating new workflow for group ['Read', 'TOPSAR-Split']
DEBUG:pyroSAR.snap.auxil:inserting node Read
DEBUG:pyroSAR.snap.auxil:setting the source of node Read to []
DEBUG:pyroSAR.snap.auxil:inserting node TOPSAR-Split after Read
DEBUG:pyroSAR.snap.auxil:setting the source of node TOPSAR-Split to Read


Processing subswath IW2 in VV polarization.


DEBUG:pyroSAR.snap.auxil:inserting node Write after TOPSAR-Split
DEBUG:pyroSAR.snap.auxil:setting the source of node Write to TOPSAR-Split
DEBUG:pyroSAR.snap.auxil:writing /data/tmp/sub/S1A_IW_SLC__1SDV_20230826T180527_20230826T180554_050050_0605B2_E768_tmp0.xml
DEBUG:pyroSAR.snap.auxil:creating new workflow for group ['Read(2)', 'TOPSAR-Split(2)']
DEBUG:pyroSAR.snap.auxil:inserting node Read
DEBUG:pyroSAR.snap.auxil:setting the source of node Read to []
DEBUG:pyroSAR.snap.auxil:inserting node TOPSAR-Split after Read
DEBUG:pyroSAR.snap.auxil:setting the source of node TOPSAR-Split to Read
DEBUG:pyroSAR.snap.auxil:inserting node Write after TOPSAR-Split
DEBUG:pyroSAR.snap.auxil:setting the source of node Write to TOPSAR-Split
DEBUG:pyroSAR.snap.auxil:writing /data/tmp/sub/S1A_IW_SLC__1SDV_20230814T180526_20230814T180553_049875_05FFB4_7D92_tmp1.xml
DEBUG:pyroSAR.snap.auxil:creating new workflow for group ['Apply-Orbit-File']
DEBUG:pyroSAR.snap.auxil:inserting node Read
DEBUG:pyroSAR.snap

Removing dark edges after terrain correction


DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f27b15ad710>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f27af061870> options
DEBUG:rasterio._env:Stopped GDALEnv <rasterio._env.GDALEnv object at 0x7f27af061870>.
DEBUG:rasterio.env:Exiting outermost env
DEBUG:rasterio.env:Exited env context: <rasterio.env.Env object at 0x7f27b15ad710>
DEBUG:pyroSAR.snap.auxil:writing /data/tmp/graph_coh.xml
INFO:pyroSAR.snap.auxil:executing node sequences..
DEBUG:pyroSAR.snap.auxil:creating new workflow for group ['Read', 'TOPSAR-Split']
DEBUG:pyroSAR.snap.auxil:inserting node Read
DEBUG:pyroSAR.snap.auxil:setting the source of node Read to []
DEBUG:pyroSAR.snap.auxil:inserting node TOPSAR-Split after Read
DEBUG:pyroSAR.snap.auxil:setting the source of node TOPSAR-Split to Read


...

DEBUG:pyroSAR.snap.auxil:inserting node Write after TOPSAR-Split
DEBUG:pyroSAR.snap.auxil:setting the source of node Write to TOPSAR-Split
DEBUG:pyroSAR.snap.auxil:writing /data/tmp/sub/S1A_IW_SLC__1SDV_20230826T180527_20230826T180554_050050_0605B2_E768_tmp0.xml
DEBUG:pyroSAR.snap.auxil:creating new workflow for group ['Read(2)', 'TOPSAR-Split(2)']
DEBUG:pyroSAR.snap.auxil:inserting node Read
DEBUG:pyroSAR.snap.auxil:setting the source of node Read to []
DEBUG:pyroSAR.snap.auxil:inserting node TOPSAR-Split after Read
DEBUG:pyroSAR.snap.auxil:setting the source of node TOPSAR-Split to Read
DEBUG:pyroSAR.snap.auxil:inserting node Write after TOPSAR-Split
DEBUG:pyroSAR.snap.auxil:setting the source of node Write to TOPSAR-Split
DEBUG:pyroSAR.snap.auxil:writing /data/tmp/sub/S1A_IW_SLC__1SDV_20230814T180526_20230814T180553_049875_05FFB4_7D92_tmp1.xml
DEBUG:pyroSAR.snap.auxil:creating new workflow for group ['Apply-Orbit-File']
DEBUG:pyroSAR.snap.auxil:inserting node Read
DEBUG:pyroSAR.snap

Removing dark edges after terrain correction


DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f27b037d410>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f27b115beb0> options
DEBUG:rasterio._env:Stopped GDALEnv <rasterio._env.GDALEnv object at 0x7f27b115beb0>.
DEBUG:rasterio.env:Exiting outermost env
DEBUG:rasterio.env:Exited env context: <rasterio.env.Env object at 0x7f27b037d410>
DEBUG:rasterio.env:Entering env context: <rasterio.env.Env object at 0x7f27b037d410>
DEBUG:rasterio.env:Starting outermost env
DEBUG:rasterio.env:No GDAL environment exists
DEBUG:rasterio.env:New GDAL environment <rasterio._env.GDALEnv object at 0x7f27b043ec20> created
DEBUG:rasterio._env:GDAL_DATA found in environment.
DEBUG:rasterio._env:PROJ_DATA found in environment.
DEBUG:rasterio._env:Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x7f27b043ec20>.
DEBUG:rasterio.env:Entered env context: <rasterio.env.Env object at 0x7f27b037d410>
DEBUG:rasterio._base:Sharing flag: 0
DEBUG:rasterio._base:Nodat

Merging and cropping current subswath


DEBUG:rasterio._io:Output nodata value read from file: 0.0
DEBUG:rasterio._io:Output nodata values: [0.0]
DEBUG:rasterio._io:all_valid: False
DEBUG:rasterio._io:mask_flags: ([<MaskFlags.nodata: 8>],)
DEBUG:rasterio._io:Jump straight to _read()
DEBUG:rasterio._io:Window: Window(col_off=0.0, row_off=0.0, width=7860.0, height=3701.0)
DEBUG:rasterio._io:IO window xoff=0.0 yoff=0.0 width=7860.0 height=3701.0
DEBUG:rasterio._io:IO window xoff=0.0 yoff=0.0 width=7860.0 height=3701.0
DEBUG:rasterio._io:Output nodata value read from file: 0.0
DEBUG:rasterio._io:Output nodata values: [0.0]
DEBUG:rasterio._io:all_valid: False
DEBUG:rasterio._io:mask_flags: ([<MaskFlags.nodata: 8>],)
DEBUG:rasterio._io:Jump straight to _read()
DEBUG:rasterio._io:Window: Window(col_off=0.0, row_off=0.0, width=7160.0, height=4461.0)
DEBUG:rasterio._io:IO window xoff=0.0 yoff=0.0 width=7160.0 height=4461.0
DEBUG:rasterio._io:IO window xoff=0.0 yoff=0.0 width=7160.0 height=4461.0
DEBUG:rasterio.env:Entering env contex

In [ ]:
from pyroSAR.snap.auxil import execute, groupbyWorkers, gpt

# grp = groupbyWorkers('./TOPSAR Coreg Interferogram IW2 to GeoTiff.xml', n=1)
grp = groupbyWorkers("./experiment_from_snap_graph.xml", n=1)
# grp = groupbyWorkers('./Coh_tmp_prep_graph_dbg.xml', n=1)

gpt("./experiment_from_snap_graph.xml", groups=grp, tmpdir="/data/tmp/")
# gpt('./Coh_tmp_prep_graph_dbg.xml', groups=grp, tmpdir='/data/tmp/')
# execute('./Coh_tmp_prep_graph_dbg.xml', gpt_args=gpt_paras)
# execute('./experiment_from_snap_graph.xml', gpt_args=gpt_paras)

In [ ]:
import rasterio as rio

# with rio.open('/data/tmp/S1_A_relOrb_103_COH__VV_20230826T180527_20230814T180526/S1_A_relOrb_103_COH__VV_20230826T180527_20230814T180526.tif') as ds:
with rio.open("/data/S1/test_ifg.tif") as ds:
    # with rio.open('/data/tmp/S1_relOrb_TEST_split_before_orbit.tif') as ds:
    print(ds.profile)
    img = ds.read(3)
    re = ds.read(1)
    im = ds.read(2)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
plt.imshow(img[::8, ::8], interpolation="nearest")  # , vmin=0, vmax=100000)
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
plt.imshow(
    np.angle(re + 1j * im)[::8, ::8], interpolation="nearest", vmin=0, vmax=100000000
)
plt.colorbar()

In [ ]:
plt.hist(img.ravel()[::100])